In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil

import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
#os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import metrics as metrics
from optimizers import *
import schedulers as schedulers

import training_utils_enformer_performer as training_utils

import enformer_performer as enformer_performer

from scipy import stats


2023-02-06 17:05:34.941819: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 17:05:35.141165: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 17:05:35.141196: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-06 17:05:36.081322: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
"gs://picard-testing-176520/enformer_baseline_paired_ATAC_RAMPAGE/tfrecords"

In [ ]:
def deserialize_tr(serialized_example,input_length,max_shift):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
    }
    ### stochastic sequence shift and gaussian noise

    rev_comp = tf.math.round(g.uniform([], 0, 1))

    shift =5

    input_seq_length = input_length + max_shift


    example = tf.io.parse_example(serialized_example, feature_map)

    target = tf.io.decode_raw(example['target'], tf.float16)
    target = tf.reshape(target,
                        (1536, 62))

    dnase_atac = tf.reduce_mean(target[:,31:],axis=1,keepdims=True)


    return {'mean_acc': tf.ensure_shape(dnase_atac,
                                        [1536,1])}
                    

In [ ]:
def return_dataset(gcs_path,
                   batch,
                   input_length,
                   max_shift,
                   num_targets,
                   options,
                   num_parallel,
                   num_epoch,
                   g):
    """
    return a tf dataset object for given gcs path
    """
    wc = "*.tfr"

    list_files = (tf.io.gfile.glob(os.path.join(gcs_path,
                                                organism,
                                                "tfrecords",
                                                wc)))
    #print(list_files)
    random.shuffle(list_files)
    files = tf.data.Dataset.list_files(list_files,shuffle=True)
    
    dataset = tf.data.TFRecordDataset(files,
                                      compression_type='ZLIB',
                                      num_parallel_reads=num_parallel)
    dataset = dataset.with_options(options)

        
    dataset = dataset.map(lambda record: deserialize_tr(record,
                                                        input_length,
                                                        max_shift,
                                                        num_targets,
                                                        g),
                          deterministic=False,
                          num_parallel_calls=num_parallel)
        

    return dataset.repeat(num_epoch).batch(batch,drop_remainder=True).prefetch(1)
